In [1]:
import os

In [2]:
%pwd

'd:\\vehicle_insurance_fraud_detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\vehicle_insurance_fraud_detection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/RagalahariAkula-42/vehicle_insurance_fraud_detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="RagalahariAkula-42"
os.environ["MLFLOW_TRACKING_PASSWORD"]="7ca8771510f6ceac31f9f0fa87bdace49455316c"

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    X_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    metric_file_name: Path
    mlflow_uri: str

In [7]:
from vifd.constants import *
from vifd.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            X_test_data_path=config.X_test_data_path,
            y_test_data_path=config.y_test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            mlflow_uri="https://dagshub.com/RagalahariAkula-42/vehicle_insurance_fraud_detection.mlflow"

        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np 
import joblib
from vifd.utils.common import save_json

[2024-01-19 23:25:20,110: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-01-19 23:25:20,112: INFO: utils: NumExpr defaulting to 8 threads.]


In [10]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        f1 = f1_score(actual, pred)
        acc = accuracy_score(actual, pred)
        return f1, acc

    def log_into_mlflow(self):
        X_test = pd.read_csv(self.config.X_test_data_path)
        y_test = pd.read_csv(self.config.y_test_data_path)['fraud_reported_Y'].values
        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            f1, acc = self.eval_metrics(y_test, predicted_qualities)
            
            scores = {"f1": f1, "acc": acc}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_metric("f1", f1)
            mlflow.log_metric("acc", acc)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="LogisticRegression")
            else:
                mlflow.sklearn.log_model(model, "model")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()

except Exception as e:
    raise e

[2024-01-19 23:25:22,040: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-19 23:25:22,045: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-19 23:25:22,046: INFO: common: created directory at: artifacts]
[2024-01-19 23:25:22,047: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-19 23:25:23,728: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\Lahari\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'LogisticRegression' already exists. Creating a new version of this model...
2024/01/19 23:25:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LogisticRegression, version 3
Created version '3' of model 'LogisticRegression'.
